# Population analysis of Phacelia with outgroups removed
Structure is a standard tool for examining population genetic structure based on allele frequencies within and among populations

In [1]:
## imports
import ipyrad as ip
import ipyrad.analysis as ipa
import ipyparallel as ipp
import toytree
import toyplot
import numpy as np   # numerical library

In [2]:
#ipyrad_0.9.84 installed on the system
## print Version of ipyrad, toytree, tetrad
print("ipyrad v. {}".format(ip.__version__))
print("toytree v. {}".format(toytree.__version__))
#print("tetrad v. {}".format(tetrad.__version__))
## print Version of Python
from platform import python_version
print("Python v.", python_version())

ipyrad v. 0.9.84
toytree v. 2.0.5
Python v. 3.10.5


In [3]:
# After the cluster is running attach to it with ipyparallel
ipyclient = ipp.Client()
print(ip.cluster_info(ipyclient))

Parallel connection | Cristaria: 8 cores
None


In [4]:
## load the hdf5 data for the STRUCTURE analysis
dataclust90 = "/home/marianna/Documents/Phacelia/Phac_Assembly/min12_clust90.snps.hdf5"

### Grouping the samples to potential populations using an 'imap' based on the topology of the retrieved ML trees

In [5]:
# group individuals into populations, according to Karuna's analysis and the new topology
imap = {
    "1": ["W6368","W6376"],
    "2": ["W5599","W6027","W6028","W6037","W6078", "W6024", "W5637", "W5636"],
    "3": ["W6373"],
    "4": ["W5610", "W6374", "W6080", "W6369"],
    "5": ["W6375", "W6031","W6370"],
    "6": ["W5145", "W5612"],
    "7": ["W6001", "W6079","W6029"], 
   
}

## require that 50% of samples have data in each group
minmap = {i: 0.5 for i in imap}

In [7]:
struct = ipa.structure(
    name = "min12", #50% missing data
    data = dataclust90,
    imap = imap,
    minmap = minmap,
    mincov = 0.5,
    workdir = "/home/marianna/Documents/Phacelia/Phac_Analyses/Structure/min12_20221117"
)

Samples: 23
Sites before filtering: 19359
Filtered (indels): 1020
Filtered (bi-allel): 210
Filtered (mincov): 579
Filtered (minmap): 5685
Filtered (subsample invariant): 6133
Filtered (minor allele frequency): 0
Filtered (combined): 10771
Sites after filtering: 8587
Sites containing missing values: 5548 (64.61%)
Missing values in SNP matrix: 13587 (6.88%)
SNPs (total): 8587
SNPs (unlinked): 4200


### Running STRUCTURE and plot results
The burnin and numreps parameters determine the length of the run.

In [8]:
struct.mainparams.burnin  = 100000
struct.mainparams.numreps = 500000

## see all mainparams
print(struct.mainparams)

#see or ser extraparams
print(struct.extraparams)

burnin             100000              
extracols          0                   
label              1                   
locdata            0                   
mapdistances       0                   
markernames        0                   
markovphase        0                   
missing            -9                  
notambiguous       -999                
numreps            500000              
onerowperind       0                   
phased             0                   
phaseinfo          0                   
phenotype          0                   
ploidy             2                   
popdata            0                   
popflag            0                   
recessivealleles   0                   

admburnin           500                 
alpha               1.0                 
alphamax            10.0                
alphapriora         1.0                 
alphapriorb         2.0                 
alphapropsd         0.025               
ancestdist          0            

In [9]:
## set a range of k-values to test
kvalues = [2, 3, 4, 5, 6, 7, 8]

In [10]:
## submit batches of 10 replicates jobs for each value of k
for kpop in kvalues:
    struct.run(kpop = kpop, nreps = 10, seed = 12345, ipyclient = ipyclient)#, force = True)

[####################] 100% 2:16:00 | running 10 structure jobs 
[####################] 100% 2:47:39 | running 10 structure jobs 
[####################] 100% 3:16:33 | running 10 structure jobs 
[####################] 100% 3:49:20 | running 10 structure jobs 
[####################] 100% 4:14:27 | running 10 structure jobs 
[####################] 100% 4:43:27 | running 10 structure jobs 
[####################] 100% 5:13:25 | running 10 structure jobs 


# Analyze results: check results in evanno table
Note the K value for which deltaK is maximum


In [11]:
etable = struct.get_evanno_table(kvalues)
etable

,Nreps,lnPK,lnPPK,deltaK,estLnProbMean,estLnProbStdev
2,10,0.00,0.00,0.000,-34209.15,111.621
3,10,-4338.57,10175.00,0.687,-38547.72,14800.358
4,10,5836.43,14792.91,3.698,-32711.29,4000.077
5,10,-8956.48,20984.70,0.776,-41667.77,27049.697
6,10,12028.22,75045.62,24.041,-29639.55,3121.561
7,10,-63017.40,128063.70,1.062,-92656.95,120642.293
8,10,65046.30,0.00,0.000,-27610.65,5631.731


In [13]:
## summarize results
struct.clumppparams.m = 3                ## use largegreedy algorithm
struct.clumppparams.greedy_option = 2    ## test nrepeat possible orders
struct.clumppparams.repeats = 100000     ## number of repeats

In [14]:
qtable = struct.get_clumpp_table(kvalues)#, max_var_multiple=100.)

[K2] 10/10 results permuted across replicates (max_var=0).
[K3] 10/10 results permuted across replicates (max_var=0).
[K4] 10/10 results permuted across replicates (max_var=0).
[K5] 10/10 results permuted across replicates (max_var=0).
[K6] 10/10 results permuted across replicates (max_var=0).
[K7] 10/10 results permuted across replicates (max_var=0).
[K8] 10/10 results permuted across replicates (max_var=0).


In [15]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t953ce7fbc73d49788f39c76126730381"> 2 3 4 5 6 7 8 K (N ancestral populations) 25000 50000 75000 100000 estLnProbMean 0 10 20 30 deltaK

In [16]:
k = 6
table = struct.get_clumpp_table(k)

[K6] 10/10 results permuted across replicates (max_var=0).


In [17]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [18]:
# build barplot
canvas = toyplot.Canvas(width=1000, height=500)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="500.0px" viewBox="0 0 1000.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9a1fc27fa63b447080f7fff170e1f7a3"> W6368 W6376 W5599 W6027 W6028 W6037 W6078 W6024 W5637 W5636 W6373 W5610 W6374 W6080 W6369 W6375 W6031 W6370 W5145 W5612 W6001 W6079 W6029 0.0 0.5 1.0

In [51]:
## Plot the resulting tree

tre = toytree.tree("/home/marianna/Documents/Phacelia/Phac_Analysis/RAxML_bipartitions.pops40_clust90 (copy).phy")
rtre = tre.root(wildcard = "W6021") #branch length of 6021 was manualy changed in order to be able to see the very short branches of the ingroup

# use canvas and axes function in order use export function
canvas, axes, mark = rtre.ladderize(1).draw(
    width = 1400,
    height = 900,
    #use_edge_length = False,
    tip_labels_align = True,
    tip_labels_style={"font-size": "15px"},
    node_labels='support',
    node_sizes=0,
    node_labels_style={"font-size": "15px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-13px"
                       
                      },
    
    );

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1400.0px" height="900.0px" viewBox="0 0 1400.0 900.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5f962b352ac84ef297f4abd28dfabba8"> 56 100 97 100 100 100 51 100 91 100 100 96 43 70 96 100 28 100 70 100 79 100 Phacelia_artemisioides_W6021_out Phacelia_sinuata_W6368 Phacelia_sinuata_W6376 Phacelia_pinnatifida_W5599 Phacelia_pinnatifida_W6027 Phacelia_pinnatifida_viscosa_W6028 Phacelia_spec_nov_W6078 Phacelia_spec_W6037 Phacelia_cf_viscosa_pinnatifida_W6024 Phacelia_spec_nov_W5636 Phacelia_spec_nov_W5637 Phacelia_nana_W6373 Phacelia_pinnatifida_W5610 Phacelia_pinnatifida_W6374 Phacelia_pinnatifida_W6080 Phacelia_sanzinii_W6369 Phacelia_setigera_var_setigera_W6375 Phacelia_setigera_var_setigera_W6370 Phacelia_artemisioides_W6031 Phacelia_setigera_W5612 Phacelia_viscosa_W5145 Phacelia_pinnatifida_W6001 Phacelia_pinnatifida_W6029 Phacelia_pinnatifida_W6079

In [20]:
#Write the order of the taxa in order to arrange accordingly the structure results
myorder = ["W6368", "W6376", "W5599","W6027", "W6028", "W6078", "W6037", "W6024", "W5636", "W5637",
          "W6373","W5610", "W6374","W6080","W6369",  "W6375", "W6370", "W6031","W5612", "W5145","W6001", "W6029","W6079"]
print("custom ordering")
print(qtable[2].loc[myorder])

custom ordering
               0          1
W6368  4.000e-01  6.000e-01
W6376  4.000e-01  6.000e-01
W5599  1.100e-03  9.989e-01
W6027  0.000e+00  1.000e+00
W6028  6.000e-04  9.994e-01
W6078  0.000e+00  1.000e+00
W6037  8.000e-04  9.992e-01
W6024  0.000e+00  1.000e+00
W5636  0.000e+00  1.000e+00
W5637  0.000e+00  1.000e+00
W6373  9.000e-01  1.000e-01
W5610  9.944e-01  5.600e-03
W6374  9.999e-01  1.000e-04
W6080  9.999e-01  1.000e-04
W6369  1.000e+00  0.000e+00
W6375  9.000e-01  1.000e-01
W6370  6.000e-01  4.000e-01
W6031  6.000e-01  4.000e-01
W5612  7.000e-01  3.000e-01
W5145  7.000e-01  3.000e-01
W6001  9.000e-01  1.000e-01
W6029  9.001e-01  9.990e-02
W6079  9.000e-01  1.000e-01


### Plot all STRUCTURE results against Phylogeny

In [53]:
# get tree from RAxML results

tre = toytree.tree("/home/marianna/Documents/Phacelia/Phac_Analysis/RAxML_bipartitions.pops40_clust90 (copy).phy")
rtre = tre.root(wildcard = "W6021")

## further styling of plot with css 
style = {"stroke":toyplot.color.black, ##near_black is giving error module 'toyplot.color' has no attribute 'near_black'
         "stroke-width": 0.25}

##    y1
## x1    x2
##    y2

## built & dissect canvas into multiple cartesian areas (x1, x2, y1, y2)
c = toyplot.Canvas(width = 900, height = 700)
a1 = c.cartesian(bounds=('1%', '49%', '6.25%', '94%'))       # The tree
a2 = c.cartesian(bounds=('46.5%', '55%', '5.25%', '91%'))  # K=2
a3 = c.cartesian(bounds=('55.5%', '64%', '5.25%', '91%'))  # K=3
a4 = c.cartesian(bounds=('64.5%', '73%', '5.25%', '91%'))  # K=4
a5 = c.cartesian(bounds=('73.5%', '82%', '5.25%', '91%'))  # K=5
a6 = c.cartesian(bounds=('82.5%', '91%', '5.25%', '91%'))  # K=6
a7 = c.cartesian(bounds=('91.5%', '100%', '5.25%', '91%'))  # K=7
a1.show = False
a2.show = False
a3.show = False
a4.show = False
a5.show = False
a6.show = False
a7.show = False
## draw the tree
rtre.ladderize(1).draw(
    axes = a1,
    use_edge_lengths = True,
    tip_labels_align = True,
    tip_labels_style = {"font-size": "9px"},
    node_labels = "support",
    node_sizes = 0,
    node_labels_style={"font-size": "9px",
                       "baseline-shift": "7px",
                       "-toyplot-anchor-shift": "-8px"});

## draw the STRUCTURE bar plots
## 'along' defines plot orientation; x = vertical; y = horizontal
a2.bars(qtable[2].loc[myorder], style = style, along = 'y');
a3.bars(qtable[3].loc[myorder], style = style, along = 'y');
a4.bars(qtable[4].loc[myorder], style = style, along = 'y');
a5.bars(qtable[5].loc[myorder], style = style, along = 'y');
a6.bars(qtable[6].loc[myorder], style = style, along = 'y');
a7.bars(qtable[7].loc[myorder], style = style, along = 'y');

## add header for the bar plots
c.text(460, 23, 'K = 2', style={"font-size": "13px"})
c.text(540, 23, 'K = 3', style={"font-size": "13px"})
c.text(620, 23, 'K = 4', style={"font-size": "13px"})
c.text(700, 23, 'K = 5', style={"font-size": "13px"})
c.text(780, 23, 'K = 6', style={"font-size": "13px"})
c.text(860, 23, 'K = 7', style={"font-size": "13px"})

## add deltaK values below the bar plots
c.text(460, 645, '0.0', style={"font-size": "10px"})
c.text(540, 645, '0.7', style={"font-size": "10px"})
c.text(620, 645, '3.7', style={"font-size": "10px"})
c.text(700, 645, '0.8', style={"font-size": "10px"})
c.text(780, 645, '24.0', style={"font-size": "10px"})
c.text(860, 645, '1.1', style={"font-size": "10px"})
c.text(655, 655, 'delta <b>K</b>', style={"font-size": "10px"});

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="900.0px" height="700.0px" viewBox="0 0 900.0 700.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2eebde98170d4210ad3397625c0e54b9"> 56 100 97 100 100 100 51 100 91 100 100 96 43 70 96 100 28 100 70 100 79 100 Phacelia_artemisioides_W6021_out Phacelia_sinuata_W6368 Phacelia_sinuata_W6376 Phacelia_pinnatifida_W5599 Phacelia_pinnatifida_W6027 Phacelia_pinnatifida_viscosa_W6028 Phacelia_spec_nov_W6078 Phacelia_spec_W6037 Phacelia_cf_viscosa_pinnatifida_W6024 Phacelia_spec_nov_W5636 Phacelia_spec_nov_W5637 Phacelia_nana_W6373 Phacelia_pinnatifida_W5610 Phacelia_pinnatifida_W6374 Phacelia_pinnatifida_W6080 Phacelia_sanzinii_W6369 Phacelia_setigera_var_setigera_W6375 Phacelia_setigera_var_setigera_W6370 Phacelia_artemisioides_W6031 Phacelia_setigera_W5612 Phacelia_viscosa_W5145 Phacelia_pinnatifida_W6001 Phacelia_pinnatifida_W6029 Phacelia_pinnatifida_W6079 K = 2 K = 3 K = 4 K = 5 K = 6 K = 7 0.0 0.7 3.7 0.8 24.0 1.1 delta K

In [54]:
import toyplot.pdf
toyplot.pdf.render(c, "/home/marianna/Documents/Phacelia/Figures/Structure_RAxML_pops40clust90_20221121.pdf");